In [135]:
import re
from copy import deepcopy

In [239]:
class State:
    def __init__(self, blueprint):
        self.ore_robots = 1
        self.clay_robots = 0
        self.obsidian_robots = 0
        self.geode_robots = 0
        self.ore = 0
        self.clay = 0
        self.obsidian = 0
        self.geode = 0
        self.time = 24
        self.blueprint = blueprint
        self.max_ore = max(m.get('ore', 0) for m in self.blueprint.values())
        self.max_clay = max(m.get('clay', 0) for m in self.blueprint.values())
        self.max_obsidian = max(m.get('obsidian', 0) for m in self.blueprint.values())

    def collect_ore(self):
        self.ore += self.ore_robots

    def collect_clay(self):
        self.clay += self.clay_robots

    def collect_obsidian(self):
        self.obsidian += self.obsidian_robots

    def collect_geode(self):
        self.geode += self.geode_robots

    def build_ore_robot(self):
        self.ore -= self.blueprint['ore_robot']['ore']
        self.ore_robots += 1

    def build_clay_robot(self):
        self.ore -= self.blueprint['clay_robot']['ore']
        self.clay_robots += 1

    def build_obsidian_robot(self):
        self.ore -= self.blueprint['obsidian_robot']['ore']
        self.clay -= self.blueprint['obsidian_robot']['clay']
        self.obsidian_robots += 1
    
    def build_geode_robot(self):
        self.ore -= self.blueprint['geode_robot']['ore']
        self.obsidian -= self.blueprint['geode_robot']['obsidian']
        self.geode_robots += 1

    def can_build_ore_robot(self):
        return self.ore >= self.blueprint['ore_robot']['ore']

    def can_build_clay_robot(self):
        return self.ore >= self.blueprint['clay_robot']['ore']

    def can_build_obsidian_robot(self):
        return self.ore >= self.blueprint['obsidian_robot']['ore'] and self.clay >= self.blueprint['obsidian_robot']['clay']

    def can_build_geode_robot(self):
        return self.ore >= self.blueprint['geode_robot']['ore'] and self.obsidian >= self.blueprint['geode_robot']['obsidian']

    def collect(self):
        self.collect_ore()
        self.collect_clay()
        self.collect_obsidian()
        self.collect_geode()
        self.time -= 1

    def get_state(self):
        return (self.ore, self.clay, self.obsidian, self.geode, self.ore_robots, 
                self.clay_robots, self.obsidian_robots, self.geode_robots)

    def __repr__(self):
        return (f'Ore: {self.ore}, Clay: {self.clay}, Obsidian: {self.obsidian}, Geode: {self.geode}\n'
        + f'Ore robots: {self.ore_robots}, Clay robots: {self.clay_robots}, Obsidian robots: {self.obsidian_robots}, Geode robots: {self.geode_robots}\n'
        + f'Time: {self.time}')


def read_input(filename):
    expr = r'Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.'
    blueprints = {}
    with open(filename, 'r') as f:
        for line in f.readlines():
            m = re.search(expr, line.strip())
            blueprints[int(m[1])] = {
                'ore_robot': {'ore': int(m[2])},
                'clay_robot': {'ore': int(m[3])},
                'obsidian_robot': {'ore': int(m[4]), 'clay': int(m[5])},
                'geode_robot': {'ore': int(m[6]), 'obsidian': int(m[7])}
            }
    return blueprints

def get_next_states(state):
    states = []
    if state.can_build_ore_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_ore_robot()
        states.append(new_state)
    if state.can_build_clay_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_clay_robot()
        states.append(new_state)
    if state.can_build_obsidian_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_obsidian_robot()
        states.append(new_state)
    if state.can_build_geode_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_geode_robot()
        states.append(new_state)
    if not (state.ore >= state.max_ore or state.clay >= state.max_clay or state.obsidian >= state.max_obsidian):
        new_state = deepcopy(state)
        new_state.collect()
        states.append(new_state)
    return states

def dfs(start_state):
    s = [start_state]
    explored = set()
    max_geodes = 0
    i = 0
    while s:
        v = s.pop()

        i += 1
        if i % 1000 == 0:
            print(v)

        if v.time == 0:
            geodes = v.geode
            if geodes > max_geodes:
                max_geodes = geodes
        elif v.get_state() not in explored:
            explored.add(v.get_state())
            for state in get_next_states(v):
                s.append(state)
    return max_geodes


In [233]:
blueprints = read_input('19_testinput.txt')

In [240]:
start_state = State(blueprints[1])

In [241]:
dfs(start_state)

Ore: 5, Clay: 18, Obsidian: 21, Geode: 0
Ore robots: 4, Clay robots: 7, Obsidian robots: 4, Geode robots: 0
Time: 0
Ore: 4, Clay: 28, Obsidian: 4, Geode: 2
Ore robots: 2, Clay robots: 7, Obsidian robots: 3, Geode robots: 2
Time: 1
Ore: 6, Clay: 57, Obsidian: 15, Geode: 0
Ore robots: 4, Clay robots: 10, Obsidian robots: 2, Geode robots: 0
Time: 0
Ore: 5, Clay: 47, Obsidian: 11, Geode: 0
Ore robots: 5, Clay robots: 8, Obsidian robots: 2, Geode robots: 0
Time: 1
Ore: 13, Clay: 25, Obsidian: 5, Geode: 1
Ore robots: 5, Clay robots: 5, Obsidian robots: 3, Geode robots: 2
Time: 0
Ore: 7, Clay: 35, Obsidian: 10, Geode: 2
Ore robots: 5, Clay robots: 7, Obsidian robots: 3, Geode robots: 1
Time: 0
Ore: 11, Clay: 45, Obsidian: 8, Geode: 1
Ore robots: 7, Clay robots: 6, Obsidian robots: 2, Geode robots: 1
Time: 0
Ore: 14, Clay: 34, Obsidian: 3, Geode: 3
Ore robots: 6, Clay robots: 5, Obsidian robots: 2, Geode robots: 2
Time: 0
Ore: 11, Clay: 15, Obsidian: 16, Geode: 0
Ore robots: 4, Clay robots: 7,

KeyboardInterrupt: 